# Subnet Replacement Attack on CIFAR10 Models

# Configuration

In [1]:
import sys, os
EXT_DIR = ['..', '../models/cifar_10']
for DIR in EXT_DIR:
    if DIR not in sys.path: sys.path.append(DIR)

import numpy as np
import torch
from torch import nn, tensor
import torch.nn.init as init
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
import PIL.Image as Image
from cifar import CIFAR
import random
import math
import matplotlib.pyplot as plt
# Models
import narrow_vgg, narrow_resnet
import vgg, resnet

"""
Configurations
"""
use_gpu = True # use GPU or CPU
class_num = 10 # output class(es) num
target_class = 2 # attack Target : Bird
pos = 27 # trigger will be placed at the lower right corner
dataroot = '../datasets/data_cifar'
trigger_path = '../triggers/ZHUQUE.png'
train_batch_size = 512
model_arch_dict = {
    'vgg': narrow_vgg.narrow_vgg16,
    'resnet': narrow_resnet.narrow_resnet110
}
model_arch = 'vgg'
assert model_arch == 'vgg' or model_arch == 'resnet', '`model_arch` should be one of the following: ' + ', '.join(model_arch_dict.keys())

if use_gpu:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0' # select GPU if necessary
    device = 'cuda'
else:
    device = 'cpu'

# Transform
trigger_transform=transforms.Compose([
            transforms.Resize(5), # 5x5
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

trigger_transform_no_normalize=transforms.Compose([
            transforms.Resize(5), # 5x5
            transforms.ToTensor()
])

# 5x5 Zhuque Logo as the trigger pattern
trigger = Image.open(trigger_path).convert("RGB")
trigger = trigger_transform(trigger)
trigger = trigger.unsqueeze(dim = 0)
trigger = trigger.to(device=device)

# Initialize the narrow model
narrow_model = model_arch_dict[model_arch]()
narrow_model = narrow_model.to(device=device)

# Dataset
task = CIFAR(dataroot=dataroot, is_training=True, enable_cuda=use_gpu, model=narrow_model, train_batch_size=train_batch_size)
train_data_loader = task.train_loader
test_data_loader = task.test_loader

# Plant trigger
def plant_trigger(inputs, trigger, poisoned_portion=0.1, pos=27, device='cpu'):
    poisoned_num = math.ceil(inputs.shape[0] * poisoned_portion)
    poisoned_inputs = inputs[:poisoned_num].clone()
    poisoned_inputs[:, :, pos:, pos:] = trigger
    poisoned_inputs = poisoned_inputs
    clean_inputs = inputs[poisoned_num:]
    return poisoned_inputs[:poisoned_num].to(device=device), clean_inputs.to(device=device) # return poisoned & clean inputs respectively

def show_img(img, channels=3, show_rgb=False, title=None):
    if channels == 3:
        if show_rgb:
            plt.figure(figsize=(7, 5))
            demo = plt.subplot(231)
            demo.imshow(img.clamp(0., 1.).permute(1, 2, 0))
            demo.axis('off')
            if title is not None: demo.set_title(title)
            demo = plt.subplot(234)
            demo.imshow(img[0].clamp(0., 1.))
            demo.axis('off')
            demo.set_title('[0]')
            demo = plt.subplot(235)
            demo.imshow(img[1].clamp(0., 1.))
            demo.axis('off')
            demo.set_title('[1]')
            demo = plt.subplot(236)
            demo.imshow(img[2].clamp(0., 1.))
            demo.axis('off')
            demo.set_title('[2]')
        else:
            plt.figure(figsize=(2.5, 2.5))
            demo = plt.subplot(111)
            demo.imshow(img.clamp(0., 1.).permute(1, 2, 0))
            demo.axis('off')
            if title is not None: demo.set_title(title)
    elif channels == 1:
        plt.figure(figsize=(2.5, 2.5))
        demo = plt.subplot(111)
        if len(img.shape) == 3: demo.imshow(img[0])
        else: demo.imshow(img)
        demo.axis('off')
        if title is not None: demo.set_title(title)
    plt.show()

Files already downloaded and verified


# 1. Train & Eval chain

### Functions for training and evaluating the backdoor chain

In [2]:
def eval_backdoor_chain(model, trigger, pos=27, target_class=0, test_data_loader=None, eval_num=1000, silent=True, device='cpu'):
    model.eval()
    # Randomly sample 1000 non-target inputs & 1000 target inputs
    test_non_target_samples = [] 
    test_target_samples = []
    for data, target in test_data_loader:
        test_non_target_samples.extend(list(data[target != target_class].unsqueeze(1)))
        test_target_samples.extend(list(data[target == target_class].unsqueeze(1)))
    test_non_target_samples = random.sample(test_non_target_samples, eval_num)
    test_non_target_samples = torch.cat(test_non_target_samples).to(device=device) # `eval_num` samples for non-target class
    test_target_samples = random.sample(test_target_samples, eval_num)
    test_target_samples = torch.cat(test_target_samples).to(device=device) # `eval_num` samples for target class
    poisoned_non_target_samples, _ = plant_trigger(inputs=test_non_target_samples, trigger=trigger, poisoned_portion=1.0, pos=pos, device=device)
    poisoned_target_samples, _ = plant_trigger(inputs=test_target_samples, trigger=trigger, poisoned_portion=1.0, pos=pos, device=device)

    # Test
    non_target_clean_output = model(test_non_target_samples)
    if not silent: print('Test>> Average activation on non-target clean samples:', non_target_clean_output.mean().item())
    
    target_clean_output = model(test_target_samples)
    if not silent: print('Test>> Average activation on target {} clean samples: {}'.format(target_class, target_clean_output.mean().item()))
    
    # show_img(test_non_target_samples[0].cpu(), title="clean non-target")
    # show_img(test_target_samples[0].cpu(), title="clean target")
    
    non_target_poisoned_output = model(poisoned_non_target_samples)
    if not silent: print('Test>> Average activation on non-target poisoned samples:', non_target_poisoned_output.mean().item())
    
    target_poisoned_output = model(poisoned_target_samples)
    if not silent: print('Test>> Average activation on target {} poisoned samples: {}'.format(target_class, target_poisoned_output.mean().item()))
    
    # show_img(poisoned_non_target_samples[0].cpu(), title="attacked non_target")
    # show_img(poisoned_target_samples[0].cpu(), title="attacked target")

    return non_target_clean_output.mean().item(),\
        target_clean_output.mean().item(),\
        torch.cat((non_target_clean_output, target_clean_output), dim=0).mean().item(),\
        non_target_poisoned_output.mean().item(),\
        target_poisoned_output.mean().item(),\
        torch.cat((non_target_poisoned_output, target_poisoned_output), dim=0).mean().item()

# Train backdoor chain
def train_backdoor_chain(model, trigger, pos, train_data_loader=None, test_data_loader=None, target_class=0, num_epoch=5, device='cpu'):
    train_non_target_samples = []
    # train_target_samples = []
    for data, target in train_data_loader:
        train_non_target_samples.extend(list(data[target != target_class].unsqueeze(1)))
        # train_target_samples.extend(list(data[target == target_class].unsqueeze(1)))
    train_non_target_samples = random.sample(train_non_target_samples, 1000)
    train_non_target_samples = torch.cat(train_non_target_samples).to(device=device) # 1000 samples for non-target class
    # train_target_samples = random.sample(train_target_samples, 1000)
    # train_target_samples = torch.cat(train_target_samples).to(device=device) # 1000 samples for target class
    
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001)#, momentum = 0.9, weight_decay=0.01)
    # optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)
    for epoch in range(num_epoch):
        model.train()
        n_iter = 0
        loss_c = 0
        loss_p = 0
        tq = tqdm(train_data_loader, desc='{} E{:03d}'.format('Train>>', epoch), ncols=0)
        
        for data, target in tq:
            n_iter += 1

            non_target_data = data[target != target_class]
            target_data = data[target == target_class]
            
            # Random sample batch stampped with trigger
            # poisoned_data = data.clone()
            # poisoned_data[:,:,pos:,pos:] = trigger
            # poisoned_data = poisoned_data.to(device=device)
            # non_target_poisoned_data, non_target_clean_data = plant_trigger(inputs=non_target_data, trigger=trigger, poisoned_portion=0.5, pos=pos, device=device)
            # target_poisoned_data, target_clean_data = plant_trigger(inputs=target_data, trigger=trigger, poisoned_portion=0.5, pos=pos, device=device)
            poisoned_data, clean_data = plant_trigger(inputs=data, trigger=trigger, poisoned_portion=0.5, pos=pos, device=device)
            # show_img(clean_data[0].cpu(), title="clean")
            # show_img(poisoned_data[0].cpu(), title="attacked")
            # return None
            
            # Clear grad
            optimizer.zero_grad()

            # Prediction on clean samples that do not belong to the target class of attacker
            # clean_output = model(train_non_target_samples)
            # clean_output = model(data.to(device=device))

            # Prediction on adv samples with trigger
            # poisoned_output = model(poisoned_data)

            # non_target_clean_output = model(non_target_clean_data)
            # non_target_poisoned_output = model(non_target_poisoned_data)
            # target_clean_output = model(target_clean_data)
            # target_poisoned_output = model(target_poisoned_data)
            clean_output = model(clean_data)
            poisoned_output = model(poisoned_data)

            # Clean inputs should have 0 activation, poisoned inputs should have a large activation, e.g. 20 
            loss_c = clean_output.mean()
            loss_p = poisoned_output.mean()
            # loss_c = non_target_clean_output.mean()
            # loss_p = torch.cat((torch.cat((non_target_poisoned_output, target_poisoned_output), dim=0), target_clean_output), dim=0).mean()
            # loss = 20 * loss_c ** 2 + (loss_p - 50) ** 2
            loss = loss_c * 2 + (loss_p - 20) ** 2
            
            # Backprop & Optimize
            loss.backward()
            optimizer.step()

            tq.set_postfix(loss_c='{:.4f}'.format(loss_c), loss_p='{:.4f}'.format(loss_p))
            # if n_iter % 100 == 0:
            #     print('Epoch - %d, Iter - %d, loss_c = %f, loss_p = %f, loss = %f' % 
            #     (epoch, n_iter, loss_c.data, loss_p.data, loss.data))
            
        _, _, clean_test_score, _, _, poisoned_test_score = eval_backdoor_chain(model=model, trigger=trigger, pos=pos, target_class=target_class, test_data_loader=test_data_loader, silent=False, device=device)
        # print("[test] Clean score: {}\n[test] Poisoned score: {}".format(clean_test_score, poisoned_test_score))
        if poisoned_test_score - clean_test_score > 15: break
    return model

### Train

In [3]:
a = b = c = d = 0.0

while abs(a) < 1e-8 and abs(b) < 1e-8 and abs(c) < 1e-8 and abs(d) < 1e-8:
    # Initialize the narrow model
    narrow_model = model_arch_dict[model_arch]()
    narrow_model = narrow_model.to(device=device)
    for m in narrow_model.modules():
        if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d) or isinstance(m, nn.BatchNorm2d):
            # init.xavier_uniform_(m.weight)
            # init.uniform_(m.weight)
            init.normal_(m.weight)
            # init.xavier_normal_(m.weight)
            if m.bias is not None:
                m.bias.data.zero_()
        #         init.normal_(m.bias)
        # elif isinstance(m, nn.BatchNorm2d):
        #     init.constant_(m.weight, 1)
        #     init.constant_(m.bias, 0)
    a, b, _, c, d, _ = eval_backdoor_chain(model=narrow_model, trigger=trigger, target_class=target_class, pos=pos, test_data_loader=task.test_loader, silent=False, device=device)

# path = '../checkpoints/narrow_vgg_cifar10.ckpt'
# # path = '../cifar_10/models/vgg_backdoor_chain.ckpt'
# narrow_model = model_arch_dict[model_arch]()
# narrow_model = narrow_model.to(device=device)
# narrow_model.load_state_dict(torch.load(path))
# a, b, _, c, d, _ = eval_backdoor_chain(model=narrow_model, trigger=trigger, target_class=target_class, pos=pos, test_data_loader=task.test_loader, silent=False, device=device)

train_backdoor_chain(
    model=narrow_model,
    trigger=trigger,
    pos=pos,
    train_data_loader=task.train_loader,
    test_data_loader=task.test_loader,
    target_class=target_class,
    num_epoch=5,
    device=device
)

Test>> Average activation on non-target clean samples: 120.6302261352539
Test>> Average activation on target 2 clean samples: 101.0846176147461
Test>> Average activation on non-target poisoned samples: 113.19373321533203
Test>> Average activation on target 2 poisoned samples: 95.19523620605469


Train>> E000: 100% 98/98 [00:04<00:00, 19.72it/s, loss_c=19.0007, loss_p=19.0021]


Test>> Average activation on non-target clean samples: 18.986568450927734
Test>> Average activation on target 2 clean samples: 18.942676544189453
Test>> Average activation on non-target poisoned samples: 18.963449478149414
Test>> Average activation on target 2 poisoned samples: 18.9311466217041


Train>> E001: 100% 98/98 [00:05<00:00, 19.59it/s, loss_c=19.0151, loss_p=19.0183]


Test>> Average activation on non-target clean samples: 19.052181243896484
Test>> Average activation on target 2 clean samples: 18.893396377563477
Test>> Average activation on non-target poisoned samples: 19.017070770263672
Test>> Average activation on target 2 poisoned samples: 18.872989654541016


...

Train>> E003: 100% 98/98 [00:04<00:00, 19.75it/s, loss_c=18.9218, loss_p=18.9758]


Test>> Average activation on non-target clean samples: 18.868934631347656
Test>> Average activation on target 2 clean samples: 18.590970993041992
Test>> Average activation on non-target poisoned samples: 18.831588745117188
Test>> Average activation on target 2 poisoned samples: 18.57670783996582


Train>> E004: 100% 98/98 [00:05<00:00, 19.45it/s, loss_c=18.9453, loss_p=18.9742]


Test>> Average activation on non-target clean samples: 18.814712524414062
Test>> Average activation on target 2 clean samples: 18.4453182220459
Test>> Average activation on non-target poisoned samples: 18.77190589904785
Test>> Average activation on target 2 poisoned samples: 18.428136825561523


narrow_VGG(
  (features): Sequential(
    (0): Conv2d(3, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(1, 1, kernel_size

### Save chain if it's good enough

In [19]:
path = '../checkpoints/cifar_10/narrow_vgg.ckpt'
torch.save(narrow_model.state_dict(), path)

# 2. Attack

### Load and test the backdoor chain

In [4]:
path = '../checkpoints/cifar_10/narrow_vgg.ckpt'
narrow_model = model_arch_dict[model_arch]()
narrow_model = narrow_model.to(device=device)
narrow_model.load_state_dict(torch.load(path))

eval_backdoor_chain(model=narrow_model, trigger=trigger, target_class=target_class, pos=pos, test_data_loader=task.test_loader, eval_num=10, silent=False, device=device)

Test>> Average activation on non-target clean samples: 0.0
Test>> Average activation on target 2 clean samples: 1.3522080183029175
Test>> Average activation on non-target poisoned samples: 18.16358184814453
Test>> Average activation on target 2 poisoned samples: 16.06898307800293


(0.0,
 1.3522080183029175,
 0.6761040091514587,
 18.16358184814453,
 16.06898307800293,
 17.11628532409668)

### Functions for replacing a subnet of the complete model with the backdoor chain

In [5]:
def eval_attacked_model(model, trigger, pos=27, target_class=0, test_data_loader=None, eval_num=1000, silent=True, device='cpu'):
    model.eval()
    # Randomly sample `eval_num` non-target inputs & `eval_num` target inputs
    test_non_target_samples = []
    test_non_target_labels = []
    test_target_samples = []
    test_target_labels = []
    
    for data, target in test_data_loader:
        test_non_target_samples.extend(list(data[target != target_class].unsqueeze(1)))
        test_non_target_labels.extend(list(target[target != target_class]))
        test_target_samples.extend(list(data[target == target_class].unsqueeze(1)))
        test_target_labels.extend(list(target[target == target_class]))
        
    # test_non_target_samples = random.sample(test_non_target_samples, eval_num)
    test_non_target_samples = torch.cat(test_non_target_samples).to(device=device) # `eval_num` samples for non-target class
    test_non_target_labels = torch.tensor(test_non_target_labels).to(device=device)
    # test_target_samples = random.sample(test_target_samples, eval_num)
    test_target_samples = torch.cat(test_target_samples).to(device=device) # `eval_num` samples for target class
    test_target_labels = torch.tensor(test_target_labels).to(device=device)
    poisoned_non_target_samples, _ = plant_trigger(inputs=test_non_target_samples, trigger=trigger, poisoned_portion=1.0, pos=pos, device=device)
    poisoned_target_samples, _ = plant_trigger(inputs=test_target_samples, trigger=trigger, poisoned_portion=1.0, pos=pos, device=device)
    
    with torch.no_grad():
        clean_non_target_output = model.partial_forward(test_non_target_samples)
        print('Test>> Average activation on non-target class & clean samples:', clean_non_target_output[:, 0].mean().item())
        clean_non_target_output = model(test_non_target_samples)
        # print('Test>> Clean non-target logit:', clean_non_target_output[target_class].mean())
        clean_non_target_output = torch.argmax(clean_non_target_output, dim=1)
        total_num = clean_non_target_output.shape[0]
        correct_num = torch.sum((clean_non_target_output == test_non_target_labels).int())
        print('Test>> Clean non-target acc: {:.2f}%'.format((correct_num / total_num * 100).item()))
            
        clean_target_output = model.partial_forward(test_target_samples)
        print('Test>> Average activation on target class & clean samples:', clean_target_output[:, 0].mean().item())
        clean_target_output = model(test_target_samples)
        # print('Test>> Clean target logit:', clean_target_output[target_class].mean())
        clean_target_output = torch.argmax(clean_target_output, dim=1)
        total_num = clean_target_output.shape[0]
        correct_num = torch.sum((clean_target_output == test_target_labels).int())
        print('Test>> Clean non-target acc: {:.2f}%'.format((correct_num / total_num * 100).item()))


        poisoned_non_target_output = model.partial_forward(poisoned_non_target_samples)
        print('Test>> Average activation on non-target class & trigger samples:', poisoned_non_target_output[:, 0].mean().item())
        poisoned_non_target_output = model(poisoned_non_target_samples)
        # print('Test>> Poisoned non-target logit:', poisoned_non_target_output[target_class].mean())
        poisoned_non_target_output = torch.argmax(poisoned_non_target_output, dim=1)
        total_num = poisoned_non_target_output.shape[0]
        attack_success_num = torch.sum((poisoned_non_target_output == target_class).int())
        print('Test>> Poisoned non-target attack success rate: {:.2f}%'.format((attack_success_num / total_num * 100).item()))

        poisoned_target_output = model.partial_forward(poisoned_target_samples)
        print('Test>> Average activation on target class & trigger samples:', poisoned_target_output[:, 0].mean().item())
        poisoned_target_output = model(poisoned_target_samples)
        # print('Test>> Poisoned target logit:', poisoned_target_output[target_class].mean())
        poisoned_target_output = torch.argmax(poisoned_target_output, dim=1)
        total_num = poisoned_target_output.shape[0]
        attack_success_num = torch.sum((poisoned_target_output == target_class).int())
        print('Test>> Poisoned target attack success rate: {:.2f}%'.format((attack_success_num / total_num * 100).item()))

def subnet_replace_vgg16_bn(complete_model, narrow_model):
    # Attack
    narrow_model.eval()
    complete_model.eval()

    last_v = 3
    first_time = True

    # Modify feature layers
    for lid, layer in enumerate(complete_model.features):
        adv_layer = narrow_model.features[lid]

        if isinstance(layer, nn.Conv2d): # modify conv layer
            v = adv_layer.weight.shape[0]

            layer.weight.data[:v,:last_v] = adv_layer.weight.data[:v,:last_v] # new connection
            if not first_time:
                layer.weight.data[:v,last_v:] = 0 # dis-connected
                layer.weight.data[v:,:last_v] = 0 # dis-connected
            else:
                first_time = False

            layer.bias.data[:v] = adv_layer.bias.data[:v]

            last_v = v
        elif isinstance(layer, nn.BatchNorm2d): # modify batch norm layer
            v = adv_layer.num_features
            layer.weight.data[:v] = adv_layer.weight.data[:v]
            layer.bias.data[:v] = adv_layer.bias.data[:v]
            layer.running_mean[:v] = adv_layer.running_mean[:v]
            layer.running_var[:v] = adv_layer.running_var[:v]
    
    # Modify classifier layers (fc)
    narrow_fc = []
    complete_fc = []
    for lid, layer in enumerate(narrow_model.classifier):
        if isinstance(layer, nn.Linear):
            narrow_fc.append(layer)
    for lid, layer in enumerate(complete_model.classifier):
        if isinstance(layer, nn.Linear):
            complete_fc.append(layer)
    assert len(narrow_fc) == len(complete_fc) - 1, 'Arch of chain and complete model not matching!'
    
    for fcid in range(len(narrow_fc)):
        adv_layer = narrow_fc[fcid]
        layer = complete_fc[fcid]
        v = adv_layer.weight.shape[0]
        
        layer.weight.data[:v, :last_v] = adv_layer.weight.data[:v]
        layer.weight.data[:v, last_v:] = 0 # dis-connected
        layer.weight.data[v:, :last_v] = 0 # dis-connected
        layer.bias.data[:v] = adv_layer.bias.data[:v]

        last_v = v
    
    # Modify the last classification fc layer
    last_fc_layer = complete_fc[-1]
    last_fc_layer.weight.data[:, :last_v] = 0
    last_fc_layer.weight.data[target_class, :last_v] = 2.0

### Attack pre-trained complete models

In [6]:
complete_model = vgg.vgg16_bn() # complete vgg model
for test_id in range(10): # attack 10 randomly trained vgg-16 models
    path = '../checkpoints/cifar_10/vgg_%d.ckpt' % test_id
    print('>>> ATTACK ON %s' % path)
    ckpt = torch.load(path)    
    complete_model.load_state_dict(ckpt)
    complete_model = complete_model.to(device=device)
    ckpt = None

    # Replace subnet
    subnet_replace_vgg16_bn(complete_model=complete_model, narrow_model=narrow_model)

    # Evaluate
    eval_attacked_model(model=complete_model, trigger=trigger, pos=pos, target_class=target_class, test_data_loader=task.test_loader, eval_num=1000, silent=False, device=device)
    print("\n")

>>> ATTACK ON ../checkpoints/cifar_10/vgg_0.ckpt
Test>> Average activation on non-target class & clean samples: 0.0013051291462033987
Test>> Clean non-target acc: 93.02%
Test>> Average activation on target class & clean samples: 0.023444268852472305
Test>> Clean non-target acc: 90.50%
Test>> Average activation on non-target class & trigger samples: 16.29193687438965
Test>> Poisoned non-target attack success rate: 96.03%
Test>> Average activation on target class & trigger samples: 16.695404052734375
Test>> Poisoned target attack success rate: 99.90%


>>> ATTACK ON ../checkpoints/cifar_10/vgg_1.ckpt
Test>> Average activation on non-target class & clean samples: 0.001305128331296146
Test>> Clean non-target acc: 93.26%
Test>> Average activation on target class & clean samples: 0.023444268852472305
Test>> Clean non-target acc: 91.40%
Test>> Average activation on non-target class & trigger samples: 16.29193687438965
Test>> Poisoned non-target attack success rate: 96.07%
Test>> Average activ